In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

import mplcyberpunk
plt.style.use('cyberpunk')

**Гипотеза компактности (для классификации)**:
Близкие объекты, как правило, лежат в одном классе.

**Гипотеза непрерывности (для регрессии)**: Близким
объектам соответствуют близкие ответы.

Метод k ближайших соседей (kNN — k nearest neighbours) метрический алгоритм для классификации объектов, основанный на оценивании сходства объектов.
Классифицируемый объект относится к тому классу, которому принадлежат ближайшие к нему объекты обучающей выборки.

Алгоритм:

1 Вычислить расстояние до каждого из объектов обучающей выборки

2 Отобрать $k$ объектов обучающей выборки, расстояние до которых минимально

3 Класс классифицируемого объекта — это класс, наиболее часто встречающийся среди $k$ ближайших соседей

**Достоинства:**
* Простота реализации.
* Классификацию, проведенную алгоритмом, легко интерпретировать путем предъявления пользователю нескольких ближайших объектов.

**Недостатки:**
* Необходимость хранения обучающей выборки целиком.
* Поиск ближайшего соседа предполагает сравнение классифицируемого объекта со всеми объектами выборки

**Выбор $k$**

Малые значения $k$ приведут к тому, что “шум” (выбросы) будет существенно влиять на результаты.

Большие значения усложняют вычисления и искажают логику ближайших соседей, в соответствии с которой ближайшие точки могут принадлежать
одному классу (гипотеза компактности).

Эвристика: $k=\sqrt n$

# Пример использования

In [7]:
url='https://drive.google.com/file/d/1fxO-zxmRcz7CTlN8o55ahqGttUfxRkiI/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

In [8]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
X = df.drop('Outcome',axis=1).values
y = df['Outcome'].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,
                                                 random_state=42, stratify=y)

In [11]:
param_grid = dict(n_neighbors=np.arange(1, 3 * int(np.ceil(np.sqrt(len(X_train[0]))))))
print(param_grid)

{'n_neighbors': array([1, 2, 3, 4, 5, 6, 7, 8])}


In [12]:
knn = KNeighborsClassifier(n_neighbors=3)

In [13]:
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False)
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(n_neighbors=3),
             param_grid={'n_neighbors': array([1, 2, 3, 4, 5, 6, 7, 8])},
             scoring='accuracy')

In [14]:
grid_mean_scores = grid.cv_results_['mean_test_score']
print(grid_mean_scores)

[0.69460784 0.71368778 0.67899698 0.7061086  0.69826546 0.70987934
 0.71772247 0.70422323]


In [15]:
plt.plot(param_grid['n_neighbors'], grid_mean_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')

Text(0, 0.5, 'Cross-Validated Accuracy')

In [16]:
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

0.7177224736048264
{'n_neighbors': 7}
KNeighborsClassifier(n_neighbors=7)


# Задание
Реализовать самостоятельно метод классификации kNN, где использовалась бы метрика Евклидового расстояния, $k=2$.

**Я так понял, что самостоятельно = без использования готовых библиотек**

*Вычисление евклидова расстояния между двумя точками*

In [20]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

*Реализация k-ближайших соседей с k=2*

In [22]:
def knn_predict(X_train, y_train, X_test, k=2):
    predictions = []
    for test_point in X_test:
        distances = []
        for i, train_point in enumerate(X_train):
            d = euclidean_distance(test_point, train_point)
            distances.append((d, y_train[i]))
        distances.sort(key=lambda x: x[0])
        nearest_neighbors = distances[:k]
        neighbor_labels = [label for _, label in nearest_neighbors]
        predicted_label = Counter(neighbor_labels).most_common(1)[0][0]
        predictions.append(predicted_label)
    return np.array(predictions)

*Пример данных (генерируем случайно)*

In [24]:
n_train = 10      # число обучающих образцов
n_test = 5        # число тестовых образцов
n_features = 2    # число признаков

In [25]:
X_train = np.random.rand(n_train, n_features)
y_train = np.random.choice(["Класс1", "Класс2"], size=n_train)
X_test = np.random.rand(n_test, n_features)

predictions = knn_predict(X_train, y_train, X_test, k=2)

print("Обучающие данные (X_train):")
print(X_train)
print("\nМетки обучающих данных (y_train):")
print(y_train)
print("\nТестовые данные (X_test):")
print(X_test)
print("\nПредсказанные метки для тестовых данных:")
print(predictions)

Обучающие данные (X_train):
[[0.63737084 0.71478098]
 [0.85468559 0.60839342]
 [0.40084563 0.38924671]
 [0.99211769 0.95098835]
 [0.819169   0.95612429]
 [0.08742194 0.46681161]
 [0.53397183 0.76342751]
 [0.96974738 0.99315026]
 [0.81119711 0.76440339]
 [0.96989626 0.62635169]]

Метки обучающих данных (y_train):
['Класс2' 'Класс1' 'Класс1' 'Класс2' 'Класс1' 'Класс2' 'Класс2' 'Класс2'
 'Класс1' 'Класс1']

Тестовые данные (X_test):
[[0.6389387  0.91380595]
 [0.868286   0.21856093]
 [0.85531196 0.5066341 ]
 [0.91665634 0.39424173]
 [0.39731483 0.62654972]]

Предсказанные метки для тестовых данных:
['Класс2' 'Класс1' 'Класс1' 'Класс1' 'Класс2']


# GitHub: [ссылка](https://github.com/Akim-norfeg)